In [1]:
import pandas as pd

In [6]:
data = pd.read_csv('data/anpc_final.csv')

/Users/luismiguel/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573971 entries, 0 to 573970
Data columns (total 15 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   Numero                                  573971 non-null  object 
 1   DataOcorrencia                          573966 non-null  object 
 2   DataFechoOperacional                    418785 non-null  object 
 3   Natureza                                573966 non-null  object 
 4   EstadoOcorrencia                        573966 non-null  object 
 5   Distrito                                573966 non-null  object 
 6   Concelho                                573966 non-null  object 
 7   Freguesia                               573964 non-null  object 
 8   Localidade                              573964 non-null  object 
 9   Latitude                                573962 non-null  object 
 10  Longitude                               5739